In [236]:
from datetime import datetime, timedelta
import re
import pandas as pd

# Let's re-load rides using what's in the JSON file to confirm that it's what we need.
# We can also re-run from this point onward without having to hit the Uber site again.
# Since the jupyter notebook is running from the root of this project, I use relative 
# pathing from there.
rides_df = pd.read_json('data/rides_raw.json')

# Let's see what the ride data looks like
print("# Dataframe details:\n")
print(rides_df.info())
print("\n# Sample data:\n")
print(rides_df.head(3))
print("\n# Sample routing value:\n")
print(rides_df.loc[0, 'routing'])
print("\n# Sample tripMetaData value:\n")
print(rides_df.loc[0, 'tripMetaData'])
print("\n# Sample breakdownDetails value:\n")
print(rides_df.loc[0, 'breakdownDetails'])


# Dataframe details:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4685 entries, 0 to 4684
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   uuid              4685 non-null   object
 1   recognizedAt      4685 non-null   int64 
 2   activityTitle     4685 non-null   object
 3   formattedTotal    4685 non-null   object
 4   routing           4685 non-null   object
 5   breakdownDetails  2275 non-null   object
 6   tripMetaData      4594 non-null   object
 7   type              4685 non-null   object
 8   status            4594 non-null   object
dtypes: int64(1), object(8)
memory usage: 329.5+ KB
None

# Sample data:

                                   uuid  recognizedAt activityTitle  \
0  d3096d6c-02bd-4f8e-855b-117588b27910    1673809557       Comfort   
1  89a3e777-2000-44af-8509-765b157dfe9e    1673808331         UberX   
2  78616fc5-6214-4f31-89a0-4229e35f0c5c    1673807621         UberX   

  forma

I'll just drop the `routing` column since that doesn't contain any valuable trip information, and the `uuid` column since that only identifies trips on the Uber end.

It's odd that there are 4594 records that have a `tripMetaData` value but only 2275 records with `breakdownDetails`. 

The value in `tripMetaData` contains valuable information such as:

1. duration
2. distance
3. pickup address
4. dropoff address
5. a URL to a map hosted by Uber (which we'll discard)

The value in `breakdownDetails` seems to be:

1. tip earnings
2. surge earnings

We already have the total earnings for each trip in the `formattedTotal` value at the root level. 

Let's look at which data *doesn't* have `breakdownDetails` values and see if there's a pattern.

In [237]:
# We need that sweet sweet trip metadata so drop any records that don't have it
rides_df = rides_df[rides_df['tripMetaData'].notnull()]
# Drop the useless routing and uuid columns
rides_df.drop(['routing', 'uuid'], axis=1, inplace=True)

# Count how many records have each number of keys and sort by key count
num_breakdown_keys = rides_df['breakdownDetails'].apply(lambda x: len(x) if isinstance(x, dict) else 0)
print(num_breakdown_keys.value_counts().sort_index())

rides_with_breakdown = rides_df[rides_df['breakdownDetails'].notnull()]
# I think this is only included if either the tip or surge value is non-zero. 
# Let's see if any records have zero for both.
breakdown_zero_valued = rides_with_breakdown['breakdownDetails'].apply(\
    lambda d: (d['formattedTip'] is None or d['formattedTip'] == '$0.00') \
            and (d['formattedSurge'] is None or d['formattedSurge'] == '$0.00'))
print(f"Number of records where both formattedTip and formattedSurge are zero or None: {breakdown_zero_valued.sum()}")

breakdownDetails
0    2319
2    2275
Name: count, dtype: int64
Number of records where both formattedTip and formattedSurge are zero or None: 0


It appears that if there was no surge amount and no tip, then the data simply doesn't include a `breakdownDetails` value. For all the records that do have this value, we'll break out `formattedTip` and `formattedSurge` to their respective values, and if either one is `None` or if `breakdownDetails` is `None` then we'll mark the corresponding value as '$0.00'.

In [238]:
def fix_breakdown(bd):
    # If breakdownDetails is None or missing, return a default dictionary.
    if bd is None or pd.isnull(bd):
        return {'formattedTip': '$0.00', 'formattedSurge': '$0.00'}
    # Otherwise, if it's a dictionary, fix the individual keys.
    if isinstance(bd, dict):
        if bd.get('formattedTip') is None:
            bd['formattedTip'] = '$0.00'
        if bd.get('formattedSurge') is None:
            bd['formattedSurge'] = '$0.00'
    return bd

rides_df['breakdownDetails'] = rides_df['breakdownDetails'].apply(fix_breakdown)

def parse_currency_to_float(currency_str):
    clean_str = currency_str.replace('$', '').strip()
    return float(clean_str)

rides_df['tip'] = rides_df['breakdownDetails'].apply(lambda bd: parse_currency_to_float(bd['formattedTip']))
rides_df['surge'] = rides_df['breakdownDetails'].apply(lambda bd: parse_currency_to_float(bd['formattedSurge']))
rides_df.drop('breakdownDetails', axis=1, inplace=True)

print(rides_df.info())
print(rides_df.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 4594 entries, 0 to 4684
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   recognizedAt    4594 non-null   int64  
 1   activityTitle   4594 non-null   object 
 2   formattedTotal  4594 non-null   object 
 3   tripMetaData    4594 non-null   object 
 4   type            4594 non-null   object 
 5   status          4594 non-null   object 
 6   tip             4594 non-null   float64
 7   surge           4594 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 323.0+ KB
None
       recognizedAt          tip        surge
count  4.594000e+03  4594.000000  4594.000000
mean   1.707871e+09     1.393052     0.663746
std    1.289361e+07     2.574267     1.536338
min    1.673390e+09     0.000000     0.000000
25%    1.700755e+09     0.000000     0.000000
50%    1.709166e+09     0.000000     0.000000
75%    1.717594e+09     3.000000     0.000000
max    1.727

Since we've processed `breakdownDetails` into individual columns, let's do the same now for `tripMetaData` and root-level columns that need processing.

In [239]:
def parse_time_to_seconds(time_str):
    matches = re.findall(r'(\d+)\s*(hr|min|sec)', time_str)
    unit_to_seconds = {'hr': 3600, 'min': 60, 'sec': 1}
    return sum(int(value) * unit_to_seconds[unit] for value, unit in matches)

def parse_miles(miles_str):
    match = re.search(r'(\d+\.?\d*)\s*mi', miles_str)
    if match:
        return float(match.group(1))

def extract_zipcode(address):
    zip_code_pattern = re.compile(r'\b\d{5}\b')
    match = zip_code_pattern.search(address)
    if match:
        return match.group()

rides_df['duration'] = rides_df['tripMetaData'].apply(lambda md: parse_time_to_seconds(md['formattedDuration']))
rides_df['distance'] = rides_df['tripMetaData'].apply(lambda md: parse_miles(md['formattedDistance']))
rides_df['pickup_address'] = rides_df['tripMetaData'].apply(lambda md: md['pickupAddress'])
rides_df['dropoff_address'] = rides_df['tripMetaData'].apply(lambda md: md['dropOffAddress'])
rides_df.drop('tripMetaData', axis=1, inplace=True)

rides_df['pickup_zip'] = rides_df['pickup_address'].apply(lambda a: extract_zipcode(a))
rides_df['dropoff_zip'] = rides_df['dropoff_address'].apply(lambda a: extract_zipcode(a))
rides_df['earnings'] = rides_df['formattedTotal'].apply(lambda t: parse_currency_to_float(t))
rides_df.drop('formattedTotal', axis=1, inplace=True)

rides_df['ride_start'] = rides_df['recognizedAt'].apply(lambda t: datetime.fromtimestamp(t))
rides_df['ride_end'] = rides_df['ride_start'] + rides_df['duration'].apply(lambda s: timedelta(seconds=s))
rides_df.drop('recognizedAt', axis=1, inplace=True)

rides_df = rides_df.rename(columns={'activityTitle': 'ride_type', 'type': 'ride_category'})

print("Example record:\n")
print(rides_df.loc[0])
print("\nInfo on new dataframe:\n")
print(rides_df.info())


Example record:

ride_type                                            Comfort
ride_category                                           TRIP
status                                             COMPLETED
tip                                                      1.0
surge                                                    0.0
duration                                                 956
distance                                                 3.9
pickup_address     N Ashland Ave, Chicago, IL 60614-1105, US
dropoff_address          W Madison St, Chicago, IL 60612, US
pickup_zip                                             60614
dropoff_zip                                            60612
earnings                                               10.72
ride_start                               2023-01-15 14:05:57
ride_end                                 2023-01-15 14:21:53
Name: 0, dtype: object

Info on new dataframe:

<class 'pandas.core.frame.DataFrame'>
Index: 4594 entries, 0 to 4684
Data columns

In [240]:
# For each activityTitle and type, let's see how many we have:
print("Ride count for each ride_type and ride_category value:\n")
print(rides_df.pivot_table(values='ride_start', index='ride_type', columns='ride_category', fill_value=0, aggfunc='count'))

Ride count for each ride_type and ride_category value:

ride_category      CT  TRIP
ride_type                  
Business Comfort    0     8
Comfort             0   301
Connect Express     0    12
Connect Saver       1     0
Delivery            3     2
Package Express     0     3
Share              13     0
Uber Pet            0    29
UberX               0  3958
UberX Priority      0     9
UberX Share       222     0
UberXL              0    33


It appears that "Share" and "UberX Share" rides only appear with a `type` of `CT`, whereas the other ride types appear with a `type` of `TRIP`. 

I'm sure that the `ride_type` of "Connect Express", "Connect Saver", "Delivery", and "Package Express" were carrying freight rather than people, and I want to focus this analysis on passenger rides.

Let's see what the Share/UberX Share records look like.

In [241]:
share_rides = rides_df[rides_df['ride_type'].isin(['Share', 'UberX Share'])]
print(share_rides.info())
print(share_rides.head())

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 41 to 4676
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ride_type        235 non-null    object        
 1   ride_category    235 non-null    object        
 2   status           235 non-null    object        
 3   tip              235 non-null    float64       
 4   surge            235 non-null    float64       
 5   duration         235 non-null    int64         
 6   distance         235 non-null    float64       
 7   pickup_address   235 non-null    object        
 8   dropoff_address  235 non-null    object        
 9   pickup_zip       235 non-null    object        
 10  dropoff_zip      235 non-null    object        
 11  earnings         235 non-null    float64       
 12  ride_start       235 non-null    datetime64[ns]
 13  ride_end         235 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), o

In [242]:
# I know these are the ride types (activityTitle) I'm interested in
passenger_ride_types = ['Business Comfort', 'Comfort', 'Share', 'Uber Pet',\
                        'UberX', 'UberX Priority', 'UberX Share', 'UberXL']
ride_categories = ['CT', 'TRIP']
print(f"Total passenger rides: {len(rides_df[rides_df['ride_type'].isin(passenger_ride_types)])}")
# How many completed passenger rides do we have?
ride_count = len(rides_df.query('status=="COMPLETED" and ride_category in @ride_categories and ride_type in @passenger_ride_types'))
print(f"There are {ride_count} completed passenger rides in this data.")

Total passenger rides: 4573
There are 4270 completed passenger rides in this data.


In [243]:
rides_df.to_csv('data/rides.csv', index=False)

print(rides_df.head())
print(rides_df.describe())
print(rides_df.info())

  ride_type ride_category     status  tip  surge  duration  distance  \
0   Comfort          TRIP  COMPLETED  1.0   0.00       956       3.9   
1     UberX          TRIP  COMPLETED  0.0   0.00       835       2.4   
2     UberX          TRIP  COMPLETED  3.0   0.00       642       1.6   
3     UberX          TRIP  COMPLETED  3.0   3.75       713       1.5   
4     UberX          TRIP  COMPLETED  3.0   0.00      1317       5.5   

                              pickup_address  \
0  N Ashland Ave, Chicago, IL 60614-1105, US   
1       N Lincoln Ave, Chicago, IL 60613, US   
2  N Clifton Ave, Chicago, IL 60657-2224, US   
3  W Belmont Ave, Chicago, IL 60657-4511, US   
4   N Whipple St, Chicago, IL 60647-3821, US   

                               dropoff_address pickup_zip dropoff_zip  \
0          W Madison St, Chicago, IL 60612, US      60614       60612   
1       N Southport Ave, Chicago, IL 60614, US      60613       60614   
2         N Lincoln Ave, Chicago, IL 60613, US      60657  

# 🎉 CLEAN DATA ACHIEVED 🥳